# Constraint / Anti-Claim Gating Demo

Goal: Support Claim X while attenuating Claim Y contradictions using a gating vector.

Claim X: "Renewable energy deployment reduces emissions."
Claim Y (contradictory cluster): "Coal usage is rapidly expanding everywhere."

Sections:
1. Corpus & labeling
2. Embeddings
3. Query & base similarity vector
4. Gate construction (boost support, suppress contradiction)
5. Uniform vs gated lattice receipts & bundles
6. Comparison metrics (ΔH, support coverage, contradiction presence)


In [1]:
# 1. Corpus & labeling

import numpy as np

from oscillink import OscillinkLattice
from oscillink.adapters.text import embed_texts

np.random.seed(11)

support_texts = [
    "Solar farms reduce reliance on fossil fuels.",
    "Wind energy displaces coal generation.",
    "Renewable deployment curbs greenhouse emissions.",
    "Battery storage stabilizes intermittent renewables.",
]
contradict_texts = [
    "Coal usage is rapidly expanding worldwide.",
    "Emissions keep rising due to new coal plants.",
    "Coal expansion outpaces renewable adoption.",
]
neutral_texts = [
    "Smart grids balance supply and demand.",
    "Hydrogen can store surplus energy.",
    "Energy efficiency lowers consumption.",
]
texts = support_texts + contradict_texts + neutral_texts
labels = (["support"] * len(support_texts) + ["contra"] * len(contradict_texts) + ["neutral"] * len(neutral_texts))
print("Total texts:", len(texts))
for i,(t,lbl) in enumerate(zip(texts, labels)):
    print(f"{i:2d} [{lbl[:6]}] {t}")

Total texts: 10
 0 [suppor] Solar farms reduce reliance on fossil fuels.
 1 [suppor] Wind energy displaces coal generation.
 2 [suppor] Renewable deployment curbs greenhouse emissions.
 3 [suppor] Battery storage stabilizes intermittent renewables.
 4 [contra] Coal usage is rapidly expanding worldwide.
 5 [contra] Emissions keep rising due to new coal plants.
 6 [contra] Coal expansion outpaces renewable adoption.
 7 [neutra] Smart grids balance supply and demand.
 8 [neutra] Hydrogen can store surplus energy.
 9 [neutra] Energy efficiency lowers consumption.


In [2]:
# 2. Embeddings
emb = embed_texts(texts)
print("Emb shape:", emb.shape)

# 3. Query vector (embed the positive claim itself)
claim_query = "Renewable energy deployment reduces emissions."
psi = embed_texts([claim_query])[0]
psi /= (np.linalg.norm(psi) + 1e-12)

# Base cosine similarities
sims = emb @ psi
print(f"Similarity stats: min={sims.min():.3f} max={sims.max():.3f} mean={sims.mean():.3f}")

Emb shape: (10, 384)
Similarity stats: min=-0.140 max=0.083 mean=-0.005


In [3]:
# 4. Build gating vector
# Strategy: boost supporting, mildly damp neutral, strongly damp contradicting.
labels_arr = np.array(labels)

support_gate = 1.0
neutral_gate = 0.65
contradict_gate = 0.15

raw_gates = np.where(labels_arr == 1, support_gate,
               np.where(labels_arr == -1, contradict_gate, neutral_gate))

# Optional: blend with similarity to refine (higher similarity within same class gets a bit more weight)
alpha = 0.4
sims_pos = sims - sims.min()
if sims_pos.max() > 0:
    sims_pos /= sims_pos.max()
refined_gates = (1 - alpha) * raw_gates + alpha * (raw_gates * (0.5 + 0.5 * sims_pos))

print("Raw gates:", raw_gates)
print("Refined gates:", np.round(refined_gates, 3))

Raw gates: [0.65 0.65 0.65 0.65 0.65 0.65 0.65 0.65 0.65 0.65]
Refined gates: [0.65  0.625 0.592 0.633 0.592 0.627 0.52  0.583 0.586 0.583]


In [6]:
# 5. Uniform lattice (no constraint gating)
from oscillink.core.lattice import OscillinkLattice

lat_uniform = OscillinkLattice(Y=emb, lamG=0.35, lamC=0.0, lamQ=0.8)
lat_uniform.set_query(psi)  # uniform gates (all ones) implicit
receipt_uniform = lat_uniform.receipt()
print("Uniform ΔH:", receipt_uniform["deltaH_total"])

bundle_u = lat_uniform.bundle(k=6)
print("Uniform bundle (id, label, align):")
for item in bundle_u:
    i = item["id"]
    lab = labels[i]
    lab_str = f"{lab:+d}" if isinstance(lab, (int, np.integer)) else str(lab)
    print(f" U {i:02d} [{lab_str}] align={item['align']:+.3f} {texts[i][:70]}")

Uniform ΔH: 11.185103416442871
Uniform bundle (id, label, align):
 U 00 [support] align=+0.922 Solar farms reduce reliance on fossil fuels.
 U 02 [support] align=+0.915 Renewable deployment curbs greenhouse emissions.
 U 05 [contra] align=+0.919 Emissions keep rising due to new coal plants.
 U 03 [support] align=+0.920 Battery storage stabilizes intermittent renewables.
 U 09 [neutral] align=+0.914 Energy efficiency lowers consumption.
 U 01 [support] align=+0.919 Wind energy displaces coal generation.


In [7]:
# 6. Constraint-gated lattice
# Apply gates via lattice B_diag rather than scaling embeddings.
lat_gated = OscillinkLattice(Y=emb, lamG=0.35, lamC=0.0, lamQ=0.8)
lat_gated.set_query(psi, gates=refined_gates.astype(np.float32))
receipt_gated = lat_gated.receipt()
print("Gated ΔH:", receipt_gated["deltaH_total"])

bundle_g = lat_gated.bundle(k=6)
print("Gated bundle (id, label, gate, align):")
for item in bundle_g:
    i = item["id"]
    lab = labels[i]
    lab_str = f"{lab:+d}" if isinstance(lab, (int, np.integer)) else str(lab)
    print(f" G {i:02d} [{lab_str}] gate={refined_gates[i]:.2f} align={item['align']:+.3f} {texts[i][:70]}")

Gated ΔH: 5.542004585266113
Gated bundle (id, label, gate, align):
 G 00 [support] gate=0.65 align=+0.844 Solar farms reduce reliance on fossil fuels.
 G 02 [support] gate=0.59 align=+0.801 Renewable deployment curbs greenhouse emissions.
 G 05 [contra] gate=0.63 align=+0.828 Emissions keep rising due to new coal plants.
 G 03 [support] gate=0.63 align=+0.832 Battery storage stabilizes intermittent renewables.
 G 09 [neutral] gate=0.58 align=+0.793 Energy efficiency lowers consumption.
 G 01 [support] gate=0.62 align=+0.827 Wind energy displaces coal generation.


In [9]:
# 7. Metrics: support coverage, contradiction leakage, ΔH change

# Recompute id lists (ensure cells executed in order)
ids_u = [item['id'] for item in bundle_u]
ids_g = [item['id'] for item in bundle_g]

labels_arr = np.array(labels)

# Map textual labels to numeric classes for counting
label_map = {"support": 1, "contra": -1, "neutral": 0}
label_nums = np.array([label_map.get(l, 0) if isinstance(l, str) else l for l in labels])

def bundle_stats(id_list, label_arr):
    if not id_list:
        return 0, 0, 0
    sup = int(np.sum(label_arr[id_list] == 1))
    contra = int(np.sum(label_arr[id_list] == -1))
    neut = int(np.sum(label_arr[id_list] == 0))
    return sup, contra, neut

sup_u, contra_u, neut_u = bundle_stats(ids_u, label_nums)
sup_g, contra_g, neut_g = bundle_stats(ids_g, label_nums)

print("Bundle composition (support, contradiction, neutral):")
print(f"Uniform : S={sup_u} C={contra_u} N={neut_u}")
print(f"Gated   : S={sup_g} C={contra_g} N={neut_g}")

impr_support = sup_g - sup_u
leakage_reduction = contra_u - contra_g

print(f"Support improvement: {impr_support:+d}")
print(f"Contradiction leakage reduction: {leakage_reduction:+d}")
ΔH_u = receipt_uniform["deltaH_total"]
ΔH_g = receipt_gated["deltaH_total"]
print(f"ΔH change (lower better): {ΔH_u:.4f} -> {ΔH_g:.4f} ({100.0 * (ΔH_g - ΔH_u) / max(ΔH_u, 1e-9):+.1f}% change)")

Bundle composition (support, contradiction, neutral):
Uniform : S=4 C=1 N=1
Gated   : S=4 C=1 N=1
Support improvement: +0
Contradiction leakage reduction: +0
ΔH change (lower better): 11.1851 -> 5.5420 (-50.5% change)


### 8. Discussion

We demonstrated a simple constraint-aware gating mechanism:

- Constructed gates from semantic label (+ support / - contradiction / 0 neutral) with refined similarity blending.
- Applied gates multiplicatively to embeddings prior to lattice settle (preserves lattice mechanics while biasing energy landscape).
- Observed changes in:
  - Bundle composition (support coverage increase, contradiction leakage reduction).
  - Global energy (ΔH). Lower ΔH after gating indicates improved coherence under the constraint objective.

This is a minimal example; more sophisticated strategies can:
- Learn gates via convex optimization aligning with a structured constraint vector.
- Combine diffusion gating with semantic constraint gating (element-wise min or learned blend).
- Calibrate neutral/contradiction scales using validation ΔH or downstream answer accuracy.

Next notebook (04) will extend this toward hallucination suppression by penalizing unsupported high-similarity distractors.

> Tip: To integrate into production, pre-compute class-conditioned gate templates and blend per query rather than recomputing from scratch.
